In [1]:
import os
import pandas as pd
from azure.storage.filedatalake import DataLakeServiceClient
from io import BytesIO

# Ladda .env OM den inte redan är laddad
if 'STORAGE_ACCOUNT_NAME' not in os.environ:
    env_file = '.env'
    if os.path.exists(env_file):
        with open(env_file, 'r', encoding='utf-8') as f:
            for line in f:
                line = line.strip()
                if line and not line.startswith('#') and '=' in line:
                    key, value = line.split('=', 1)
                    value = value.strip().strip('"').strip("'")
                    os.environ[key.strip()] = value
        print(f"✅ Laddade .env från: {env_file}")

import config as cfg

def read_parquet_from_adls(path):
    svc = DataLakeServiceClient(
        account_url=f"https://{cfg.STORAGE_ACCOUNT_NAME}.dfs.core.windows.net",
        credential=cfg.STORAGE_ACCOUNT_KEY
    )
    fs = svc.get_file_system_client(cfg.CONTAINER_NAME)
    fc = fs.get_file_client(path)
    data = fc.download_file().readall()
    buf = BytesIO(data)
    return pd.read_parquet(buf)

df = read_parquet_from_adls("curated/planned/df_planned_latest.parquet")
df.head()

✅ Laddade .env från: .env


,AdvertisedTrainIdent,service_date,Tripdate,start_planned,end_planned,duration_minutes,start_station,end_station,stops_count,through_stations,...,ToLocations,start_operator,start_owner,start_typeoftraffic,start_deviation,start_hour,start_weekday_,start_mounth,is_weekday,distance_km
0,1,2025-10-24,2025-10-24,2025-10-24 22:56:00,2025-10-24 23:27:00,31.0,NR,LP,2,"NR,LP",...,Mc,SJ,SJ,None,None,22,4,10,1,38.017
1,1,2025-10-25,2025-10-25,2025-10-25 01:16:00,2025-10-25 04:59:00,223.0,N,MC,6,"N,AV,HM,HB,LU,MC",...,Mc,SJ,SJ,None,None,1,5,10,0,249.538
2,10,2025-10-25,2025-10-25,2025-10-25 08:52:00,2025-10-25 17:51:00,539.0,CST,DU,13,"CST,VÅ,SHV,HKL,SUC,ÅG,BÄ,ÖS,KRM,JPN,UN,ÅRE,DU",...,Du,SJ,SJ,None,None,8,5,10,0,527.675
3,1001,2025-10-25,2025-10-25,2025-10-25 01:49:00,2025-10-25 02:09:00,20.0,LU,HIE,5,"LU,BLV,MC,TRI,HIE",...,Dk.kh,TDEV,Ö-TÅG,None,None,1,5,10,0,20.757
4,1004,2025-10-25,2025-10-25,2025-10-25 03:18:00,2025-10-25 03:39:00,21.0,HIE,LU,5,"HIE,TRI,MC,BLV,LU",...,Lu,TDEV,SKANE,None,None,3,5,10,0,20.757
